In [2]:
import os

In [3]:
import datetime
import numpy as np
import pandas as pd

In [4]:
base_dir = '../data/'
# base_dir = "/home/jovyan/at102-group4/crawler_data/clean_data"

d1 = datetime.date.today().strftime("%Y-%m-%d")
# d1 = "2022-01-06"
# d1

In [5]:
d1

'2022-03-09'

In [46]:
file_name = 'final_'+d1+'.csv'
file_path = os.path.join(base_dir,file_name) 
print(file_path)
df = pd.read_csv(file_path)

../data/final_2022-03-09.csv


In [47]:
df['標題'] = df['標題'].astype(str)
df['內容'] = df['內容'].astype(str)
new_df = pd.DataFrame({'comment_text':df[['標題', '內容']].replace('\\n', '', regex=True).agg(''.join, axis=1)})
# new_df = pd.DataFrame({'comment_text':df[['內容']].replace('\\n', '', regex=True).agg(''.join, axis=1)})

In [48]:
try:
  import transformers
except:
  ! pip install -q transformers

In [49]:
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import transformers
from transformers import BertTokenizer, BertModel, BertConfig

In [50]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [51]:
tokenizer = BertTokenizer.from_pretrained('../module/bert-base-chinese')
num_added_toks = tokenizer.add_tokens(['5G','4G','3G','2G','NP','LTE','MB', 'WiFi','2CA','3CA','4CA'])

In [52]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('../module/bert-base-chinese')
        self.l1.resize_token_embeddings(len(tokenizer))
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 5)
    def forward(self, ids, mask, token_type_ids):
#         _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output
    
model = BERTClass()
# model.load_state_dict(torch.load('multi_label_1222.pth'))  # https://blog.csdn.net/Avrilzyx/article/details/114586729
model.load_state_dict(torch.load('multi_label_1222_relabel.pth', map_location=torch.device('cpu')))
model.to(device)


Some weights of the model checkpoint at ../module/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21139, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (drop

In [53]:
MAX_LEN = 200
VALID_BATCH_SIZE = 4

In [54]:
tokenizer = BertTokenizer.from_pretrained('../module/bert-base-chinese')

In [55]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
#         self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
#             'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [56]:
testing_set = CustomDataset(new_df, tokenizer, MAX_LEN)

In [57]:
test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

testing_loader = DataLoader(testing_set, **test_params)

In [58]:
from tqdm import tqdm

In [59]:
def validation():
    model.eval()
    fin_outputs=[]
    fin_topk=[]
    with torch.no_grad():
        for _, data in enumerate(tqdm(testing_loader), 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#             targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
#             fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
#             print(fin_outputs)
#             aaa = torch.sigmoid(outputs).cpu().detach().flatten()
            aaa = torch.sigmoid(outputs).cpu().detach()
#             print(aaa)
#             print(torch.topk(aaa,2))
            fin_topk.extend(torch.topk(aaa,2).indices.numpy().tolist())
#             print(torch.topk(torch.sigmoid(outputs).cpu().detach().flatten(), 2).indices)
    return fin_outputs, fin_topk

In [60]:
outputs, fin_topk = validation()

  0%|          | 0/13 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\nhsiao\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 13/13 [00:1

In [61]:
outputs = np.array(outputs)
outputs.shape

fin_topk = np.array(fin_topk)
fin_topk.shape

(50, 2)

In [62]:
outputs2 = np.array(outputs) >= 0.41

In [63]:
filter_df = pd.DataFrame(outputs2)

In [64]:
dct = {0: '資費', 1: '網速', 2: '收訊', 3:'5G', 4:'其他'}

In [65]:
def myfunction(idx, x):
    mylist = []
    if(filter_df.iloc[idx][x[0]]):
        mylist.append(dct[x[0]])
    if(filter_df.iloc[idx][x[1]]):
        mylist.append(dct[x[1]])
    return "/".join(str(x) for x in mylist)

In [66]:
result = np.array([myfunction(i,v) for i,v in enumerate(fin_topk)])

In [67]:
# result

In [68]:
# pd.DataFrame(result)

In [69]:
df['predict_tag'] = pd.DataFrame(result)

In [70]:
df

,發文者,標題,推回文類別,內容,發文時間,觀看次數,推噓評價,發文者分數,URL,filename,predict_tag
0,CharlesKSQ,Re: [問題] 中華電信合約到期不管它會怎樣嗎,主文,我上個合約是中華學專388購機綁30個月一個月有5GB流量，網內每通前10分鐘免費這個約是2...,Tue Mar 8 12:33:05 2022,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646713988...,ptt_2022-03-09.csv,資費
1,cat031147,Re: [問題] 中華電信合約到期不管它會怎樣嗎,推文,之前經驗是不管他會照原合約,03/08 15:31,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646713988...,ptt_2022-03-09.csv,其他
2,a281393,Re: [問題] 中華電信合約到期不管它會怎樣嗎,推文,我之前變成牌價超貴的...建議趕快續約,03/08 18:01,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646713988...,ptt_2022-03-09.csv,資費/其他
3,kandyson2001,Re: [問題] 中華電信合約到期不管它會怎樣嗎,推文,會變成原費率，建議可以先問一下原本是多少我之前一支門號188，沒續約後，他自動變成488詳細...,03/08 18:30,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646713988...,ptt_2022-03-09.csv,資費
4,craig,Re: [問題] 中華電信合約到期不管它會怎樣嗎,推文,學生辦學專不是更便宜？,03/09 01:51,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646713988...,ptt_2022-03-09.csv,資費
5,CharlesKSQ,Re: [問題] 中華電信合約到期不管它會怎樣嗎,推文,沒有喔樓上，學專單門號吃到飽要588，我企客只要499,03/09 07:46,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646713988...,ptt_2022-03-09.csv,資費
6,shinlure,Re: [問題] 中華電信合約到期不管它會怎樣嗎,主文,※ 引述《samlin92303 ( mimic)》之銘言：: 我的合約到今年4月，可是還沒...,Tue Mar 8 10:22:31 2022,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646706153...,ptt_2022-03-09.csv,資費/其他
7,Dheroblood,Re: [問題] 中華電信合約到期不管它會怎樣嗎,推文,不續就是延續合約？？？ 學到一招了,03/08 12:02,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646706153...,ptt_2022-03-09.csv,資費/其他
8,firenick,Re: [問題] 中華電信合約到期不管它會怎樣嗎,推文,呃，這不是正常情況吧？合約到期就是取消優惠，回覆合約牌價，不是嗎？,03/08 12:15,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646706153...,ptt_2022-03-09.csv,資費/其他
9,curlymonkey,Re: [問題] 中華電信合約到期不管它會怎樣嗎,推文,不續約就是沒有優惠原價啊,03/08 12:22,NaN,NaN,NaN,https://www.ptt.cc/bbs/MobileComm/M.1646706153...,ptt_2022-03-09.csv,資費


In [71]:
upload_file_name = 'final_' + d1 + '.csv'
upload_file_path = os.path.join(base_dir,upload_file_name)

In [72]:
upload_file_path

'../data/final_2022-03-09.csv'

In [73]:
df.to_csv(upload_file_path, index=False)

In [74]:
!./sshpass -p 'AIA@10@Fruit' scp -q -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null {upload_file_path} root@159.223.59.207:/var/www/html/Fruit/pubCache

'.' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [75]:
print("Finish")

Finish
